In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
import os
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation, Flatten, Dropout
from keras.layers import LSTM

In [ ]:
path="/gdrive/My Drive/Colab Notebooks/dacon/northPole"

In [ ]:
with open(path+"/dataP/melt.pickle", 'rb') as fr:
    melt = pickle.load(fr)

In [ ]:
def dataTrans(data):
    data_size=len(data)
    sequence=7
    now_data=[]

    for i in range(data_size):
        now_data.append(data[i].flatten())

    x_train=[]
    y_train=[]

    for i in range(data_size-sequence):
        x_train.append(np.array(now_data[i:i+sequence]))
        y_train.append(np.array(now_data[i+sequence]))
    x_train=np.array(x_train)
    y_train=np.array(y_train)

    return x_train/250,y_train/250

def make_model(x_shape1,x_shape2,y_shapeF,dropout):
    # LSTM
    model = Sequential()
    model.add(LSTM(128, input_shape=(x_shape1, x_shape2), activation='relu', return_sequences=True))
    model.add(Dropout(dropout)) 
    model.add(LSTM(64, activation="relu", return_sequences=False))
    model.add(Dropout(dropout)) 
    model.add(Dense(y_shapeF))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def predictF(x_train,y_train,pred_size):
    # predict
    pred=[]
    predict_size=pred_size
    x=np.concatenate((x_train[-1][1:],np.array([y_train[-1]])),axis=0)
    for i in range(predict_size):
        y = model.predict(np.array([x]))
        pred.append(y[0]*250)
        x=np.concatenate((x[1:],y),axis=0)

    pred=np.array(pred)
    return pred

# 전처리 및 학습

In [ ]:
if __name__ == "__main__":
    # [데이터변환]
    x_train,y_train=dataTrans(melt)

    # [모델만들기]
    dropout=0.3
    model=make_model(x_train.shape[1],x_train.shape[2],y_train.shape[-1],dropout)

    # [학습]
    # 콜백 
    checkpoint_path = path+"/models/model_{epoch}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)

    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                    save_weights_only=True,
                                                    verbose=1,
                                                    period=10) 
    # 파라미터 
    batch_size=14
    epoch=200
    verbose=1

    # 시작
    model_fit = model.fit(x_train, y_train, 
                        batch_size=batch_size, 
                        epochs=epoch,
                        verbose=verbose,
                        callbacks=[cp_callback])

    # [예측]
    pred_size=24
    subL=predictF(x_train,y_train,pred_size)

Epoch 1/200
34/34 [==============================] - 52s 1s/step - loss: 0.3092
Epoch 2/200
34/34 [==============================] - 49s 1s/step - loss: 0.0551
Epoch 3/200
34/34 [==============================] - 49s 1s/step - loss: 0.0435
Epoch 4/200
34/34 [==============================] - 49s 1s/step - loss: 0.0321
Epoch 5/200
34/34 [==============================] - 49s 1s/step - loss: 0.0355
Epoch 6/200
34/34 [==============================] - 49s 1s/step - loss: 0.0338
Epoch 7/200
34/34 [==============================] - 49s 1s/step - loss: 0.0274
Epoch 8/200
34/34 [==============================] - 49s 1s/step - loss: 0.0227
Epoch 9/200
34/34 [==============================] - 49s 1s/step - loss: 0.0227
Epoch 10/200
34/34 [==============================] - 49s 1s/step - loss: 0.0224

Epoch 00010: saving model to /gdrive/My Drive/Colab Notebooks/dacon/northPole/models/model_10.ckpt
Epoch 11/200
34/34 [==============================] - 52s 2s/step - loss: 0.0215
Epoch 12/200
34/34

In [ ]:
# 피클로 저장
with open(path+"/sub.pickle", 'wb') as fw:
    pickle.dump(subL, fw)

NameError: ignored

# 제출파일만들기

In [ ]:
# 피클 읽기
with open(path+"/sub.pickle", 'rb') as fr:
    L=pickle.load(fr)

# 제출 샘플 읽기
path="/gdrive/My Drive/Colab Notebooks/dacon/northPole"
sub_sample=pd.read_csv(path+"/sample_submission.csv")

array([-0.00264693, -0.0011997 , -0.00440951, ..., -0.00271079,
        0.0024405 , -0.00157703], dtype=float32)

In [ ]:
predSize=24

column_count=136192
df=pd.DataFrame(columns=["month"]+[str(i) for i in range(column_count)])

for i in range(predSize):
    print(i)
    df.loc[i]=np.concatenate((np.array([sub_sample["month"][i]]),L[i]),axis=0)
df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


,month,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,136152,136153,136154,136155,136156,136157,136158,136159,136160,136161,136162,136163,136164,136165,136166,136167,136168,136169,136170,136171,136172,136173,136174,136175,136176,136177,136178,136179,136180,136181,136182,136183,136184,136185,136186,136187,136188,136189,136190,136191
0,2019-01,-0.0026469256,-0.001199698,-0.0044095106,0.0018393438,0.00028345312,2.4173914e-05,-0.0075206626,0.00075781083,0.0042423853,0.00050610653,0.0063168947,0.0007096096,0.0023869865,-0.0017176117,0.00062129484,-0.00069279043,0.0001738515,-0.00088372827,-0.0015613113,-0.00072460173,0.0060987184,-0.0009826666,0.0011291099,-0.0004922387,-0.00014768167,-7.2984105e-05,0.003391895,0.00044811406,0.010235468,0.0010705363,-0.0018719283,0.0018563997,0.0020537514,0.00055561395,-0.0031610325,-0.0009826744,-0.009421536,0.0023849753,0.0001094904,...,-0.0014736159,0.0012991556,-0.00034859136,-0.0015788024,0.0055292784,0.00523632,0.002726069,0.0012959507,3.695915e-05,0.00032988304,0.0058157947,-0.00048315633,-0.0003860714,-0.00497377,-0.002986641,0.0022345707,-0.0008129033,0.0022432762,0.0011741814,-0.00011914267,0.0062613026,-0.0009659832,0.001940231,0.0026560228,-0.004092988,0.001205619,0.0042115105,-0.0024361499,0.0025312137,-0.0002394975,0.0013760801,0.0009219393,-0.0005204286,0.0016576275,-0.00032924436,-0.0006985747,0.0049326643,-0.0027107915,0.002440499,-0.0015770328
1,2019-02,-0.0023459462,-0.00080075633,0.0013839508,0.0006651904,-0.0016560389,-0.00029879858,-0.0042587654,-0.00011638531,0.002201783,-0.00039726548,0.005204427,-0.00013428007,0.002538766,-0.00041134807,0.00041323825,-0.00015948138,0.00090857066,-0.0007108767,-0.0011480305,-0.00075375993,0.0006582777,-3.240757e-06,0.00064629695,-8.979259e-05,-0.00067507755,-0.00066682376,0.0011433495,0.00017086233,0.003732446,0.0007080181,-0.0007612516,-9.000087e-05,0.0006654954,0.0010742542,0.00016886224,0.000757811,-0.006986851,0.0030232233,0.0009540108,...,-0.0012198639,0.0011461532,-0.00065784855,-0.0010643698,0.0017296533,0.003406913,0.0025195507,0.0018317342,-2.1577534e-05,-0.00020679772,0.002007704,-0.0006880991,0.0015500602,-0.0011667253,-0.00096076063,0.0054376647,-0.0012878106,0.0013790812,0.0017394405,0.00049848086,0.0037645136,0.001135325,-0.00062973093,0.00061466015,0.0021561591,-0.001071935,0.0012141891,-0.0010245749,0.00052901125,0.0036039285,0.0009163787,-0.00031747913,-0.0003844059,0.0013701088,9.285759e-05,0.00064458686,0.0003475543,-0.0010618162,0.003227582,-0.0036640149
2,2019-03,-0.0024057485,-0.00053137855,0.004496542,0.00028499842,-0.002885662,-0.000514661,-0.002389528,-0.00038520125,0.0003300629,-0.0014357283,0.0043153577,-0.0007914837,0.002455152,0.00042634475,0.00044898695,0.00020364331,0.001332264,-0.0005791587,-0.00090879103,-0.00074492476,-0.0029315634,5.3121992e-05,0.00034744153,0.0001429819,-0.00077224034,-0.0007104356,-0.00066688337,-4.086121e-05,-0.00074536365,0.0004995854,2.8027898e-05,-0.0012410574,0.00020008895,9.922587e-05,0.002371393,0.0019037961,-0.004475985,0.0025953169,0.0011432664,...,-0.0012302392,0.00096560596,-0.0008124798,-0.00074786117,-0.00073349656,0.002735719,0.0022818171,0.0017164585,-3.916685e-05,-0.0005205658,-0.00027709393,4.8592028e-05,0.0020537956,0.001548884,0.00036143273,0.0051161065,-0.0012600287,0.00093586755,0.0016130605,0.00016805802,0.0024975839,0.0011734646,-0.0014971187,-0.0010724816,0.004841142,-0.0024261908,-0.00059334433,-5.2777068e-05,-0.00086166256,0.0033329953,0.00012849523,-0.0012252004,-0.0002223269,0.0011069688,0.00045457546,0.0010699899,-0.002442497,-0.0004096961,0.002340218,-0.003332276
3,2019-04,-0.0023451685,-0.00017624233,0.0056250882,0.0007281355,-0.003463312,-0.0003327002,-0.0011821117,-0.000974445,-0.0022120676,-0.002135047,0.002955557,-0.0014996412,0.0018456946,0.00041688053,0.00059470267,0.00089215394,0.0014684263,0.00025919854,-0.0009954683,-0.001299127,-0.0050181863,-0.00046272908,0.00017360525,2.2670862e-05,-

In [ ]:
for j in range(column_count):
    df[str(j)]=df[str(j)].astype(float)

df.to_csv(f'{path}/sub/sub_1.csv',index=False)

# 학습저장

In [ ]:
checkpoint_path = path+"/models/model_{epoch}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
latest = tf.train.latest_checkpoint(checkpoint_dir)

model = make_model(x_train.shape[1],x_train.shape[2],y_train.shape[-1],dropout)
model.load_weights(latest)